# 18 - DuckDB + RNN: Series Temporales con Redes Neuronales Recurrentes

## 🎯 Objetivos
- Procesamiento de series temporales con DuckDB
- Implementar múltiples arquitecturas RNN (LSTM, GRU, Bidirectional)
- Attention mechanisms para series temporales
- Comparación de arquitecturas
- Multi-step forecasting
- Export a ONNX para producción
- MLflow tracking completo

## 📚 Tecnologías
- **DuckDB**: Procesamiento de datos
- **PyTorch**: Framework de deep learning
- **LSTM/GRU**: Arquitecturas recurrentes
- **Attention**: Mecanismos de atención
- **MLflow**: Experiment tracking
- **ONNX**: Model export

## ⭐ Complejidad: Avanzado

## 1. Instalación y Setup

In [ ]:
# Instalar dependencias
!pip install duckdb pandas numpy torch torchvision mlflow onnx onnxruntime matplotlib seaborn plotly scikit-learn -q

In [ ]:
import duckdb
import mlflow
import mlflow.pytorch
import mlflow.onnx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from pathlib import Path
import json
import time
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ONNX
import onnx
import onnxruntime as rt

# Config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sns.set_style('whitegrid')

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ DuckDB version: {duckdb.__version__}")
print(f"✅ MLflow version: {mlflow.__version__}")
print(f"✅ ONNX version: {onnx.__version__}")
print(f"✅ Device: {device}")

## 2. Configurar MLflow

In [ ]:
mlflow.set_tracking_uri("./mlruns")
experiment_name = "rnn_timeseries_forecasting"
mlflow.set_experiment(experiment_name)

print(f"✅ MLflow configurado")
print(f"📊 Experimento: {experiment_name}")

## 3. Generar Datos de Serie Temporal con DuckDB

In [ ]:
# Conectar a DuckDB
con = duckdb.connect(':memory:')

# Generar serie temporal compleja
np.random.seed(42)

# 3 años de datos por hora
n_hours = 24 * 365 * 3
start_date = datetime(2021, 1, 1)
dates = pd.date_range(start=start_date, periods=n_hours, freq='H')

# Componentes de la serie
# 1. Tendencia
trend = np.linspace(1000, 1500, n_hours)

# 2. Estacionalidad anual
annual_season = 150 * np.sin(2 * np.pi * np.arange(n_hours) / (24 * 365))

# 3. Estacionalidad semanal
weekly_season = 80 * np.sin(2 * np.pi * np.arange(n_hours) / (24 * 7))

# 4. Estacionalidad diaria
daily_season = 50 * np.sin(2 * np.pi * np.arange(n_hours) / 24)

# 5. Ruido
noise = np.random.normal(0, 30, n_hours)

# 6. Eventos especiales (picos de demanda)
special_events = np.zeros(n_hours)
# Picos aleatorios
event_indices = np.random.choice(n_hours, size=50, replace=False)
for idx in event_indices:
    duration = np.random.randint(1, 24)
    magnitude = np.random.uniform(200, 400)
    special_events[idx:min(idx+duration, n_hours)] = magnitude

# Serie final
energy_demand = trend + annual_season + weekly_season + daily_season + noise + special_events
energy_demand = np.maximum(energy_demand, 0)

# Crear DataFrame
df = pd.DataFrame({
    'timestamp': dates,
    'energy_demand': energy_demand,
    'hour': dates.hour,
    'day_of_week': dates.dayofweek,
    'day_of_month': dates.day,
    'month': dates.month,
    'year': dates.year,
    'is_weekend': dates.dayofweek.isin([5, 6]).astype(int),
    'is_business_hours': ((dates.hour >= 8) & (dates.hour <= 18)).astype(int)
})

print(f"📊 Serie temporal generada")
print(f"   Período: {df['timestamp'].min()} a {df['timestamp'].max()}")
print(f"   Total registros: {len(df):,}")
print(f"   Frecuencia: Horaria")
print(f"\n📊 Estadísticas:")
print(df['energy_demand'].describe())

## 4. Análisis Exploratorio con DuckDB

In [ ]:
# Estadísticas por hora del día
hourly_stats = con.execute("""
    SELECT 
        hour,
        COUNT(*) as observations,
        ROUND(AVG(energy_demand), 2) as avg_demand,
        ROUND(STDDEV(energy_demand), 2) as std_demand,
        ROUND(MIN(energy_demand), 2) as min_demand,
        ROUND(MAX(energy_demand), 2) as max_demand
    FROM df
    GROUP BY hour
    ORDER BY hour
""").df()

print("📊 Patrón por hora del día:")
print(hourly_stats)

# Comparación fines de semana vs días laborables
weekend_comparison = con.execute("""
    SELECT 
        CASE WHEN is_weekend = 1 THEN 'Weekend' ELSE 'Weekday' END as day_type,
        COUNT(*) as observations,
        ROUND(AVG(energy_demand), 2) as avg_demand,
        ROUND(STDDEV(energy_demand), 2) as std_demand
    FROM df
    GROUP BY is_weekend
""").df()

print("\n📊 Weekend vs Weekday:")
print(weekend_comparison)

# Tendencia mensual
monthly_trend = con.execute("""
    SELECT 
        year,
        month,
        ROUND(AVG(energy_demand), 2) as avg_demand,
        COUNT(*) as hours
    FROM df
    GROUP BY year, month
    ORDER BY year, month
""").df()

print("\n📊 Tendencia mensual (primeros 12 meses):")
print(monthly_trend.head(12))

## 5. Visualización de la Serie

In [ ]:
# Visualización interactiva
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Serie Temporal Completa', 'Último Mes', 'Patrón Semanal Promedio'),
    vertical_spacing=0.1
)

# Serie completa
fig.add_trace(
    go.Scatter(x=df['timestamp'], y=df['energy_demand'], 
               mode='lines', name='Demanda de Energía', line=dict(width=1)),
    row=1, col=1
)

# Último mes
last_month = df.tail(24*30)
fig.add_trace(
    go.Scatter(x=last_month['timestamp'], y=last_month['energy_demand'],
               mode='lines', name='Último Mes', line=dict(color='red', width=1)),
    row=2, col=1
)

# Patrón semanal
weekly_pattern = df.groupby(['day_of_week', 'hour'])['energy_demand'].mean().reset_index()
for day in range(7):
    day_data = weekly_pattern[weekly_pattern['day_of_week'] == day]
    day_names = ['Lun', 'Mar', 'Mie', 'Jue', 'Vie', 'Sab', 'Dom']
    fig.add_trace(
        go.Scatter(x=day_data['hour'], y=day_data['energy_demand'],
                   mode='lines', name=day_names[day]),
        row=3, col=1
    )

fig.update_layout(height=900, showlegend=True, title_text="Análisis de Demanda de Energía")
fig.update_xaxes(title_text="Hora", row=3, col=1)
fig.update_yaxes(title_text="Demanda (MW)", row=1, col=1)
fig.update_yaxes(title_text="Demanda (MW)", row=2, col=1)
fig.update_yaxes(title_text="Demanda (MW)", row=3, col=1)
fig.show()

print("✅ Visualizaciones generadas")

## 6. Preparar Datos para RNN

In [ ]:
def create_sequences(data, seq_length, pred_length=1):
    """
    Crea secuencias para entrenamiento de RNN
    
    Args:
        data: Serie temporal
        seq_length: Longitud de la secuencia de entrada
        pred_length: Número de pasos a predecir
    
    Returns:
        X, y: Secuencias de entrada y salida
    """
    X, y = [], []
    
    for i in range(len(data) - seq_length - pred_length + 1):
        X.append(data[i:i + seq_length])
        if pred_length == 1:
            y.append(data[i + seq_length])
        else:
            y.append(data[i + seq_length:i + seq_length + pred_length])
    
    return np.array(X), np.array(y)

# Normalizar datos
scaler = MinMaxScaler()
energy_scaled = scaler.fit_transform(df[['energy_demand']]).flatten()

# Parámetros
seq_length = 24 * 7  # 1 semana de datos (24 horas * 7 días)
pred_length = 24  # Predecir próximas 24 horas

# Crear secuencias
X, y = create_sequences(energy_scaled, seq_length, pred_length)

print(f"📊 Secuencias creadas:")
print(f"   Longitud de secuencia de entrada: {seq_length} horas (1 semana)")
print(f"   Longitud de predicción: {pred_length} horas (1 día)")
print(f"   Total secuencias: {len(X):,}")
print(f"   Shape X: {X.shape}")
print(f"   Shape y: {y.shape}")

# Split train/val/test
train_size = int(0.7 * len(X))
val_size = int(0.15 * len(X))

X_train = X[:train_size]
y_train = y[:train_size]

X_val = X[train_size:train_size + val_size]
y_val = y[train_size:train_size + val_size]

X_test = X[train_size + val_size:]
y_test = y[train_size + val_size:]

# Convertir a tensores
X_train_tensor = torch.FloatTensor(X_train).unsqueeze(-1).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)

X_val_tensor = torch.FloatTensor(X_val).unsqueeze(-1).to(device)
y_val_tensor = torch.FloatTensor(y_val).to(device)

X_test_tensor = torch.FloatTensor(X_test).unsqueeze(-1).to(device)
y_test_tensor = torch.FloatTensor(y_test).to(device)

print(f"\n📊 Splits:")
print(f"   Train: {len(X_train):,} secuencias")
print(f"   Val: {len(X_val):,} secuencias")
print(f"   Test: {len(X_test):,} secuencias")

# DataLoaders
batch_size = 32

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

print(f"\n✅ DataLoaders creados (batch_size={batch_size})")

## 7. Arquitecturas RNN

In [ ]:
# 1. LSTM Simple
class SimpleLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=24, dropout=0.2):
        super(SimpleLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # x: (batch, seq_len, input_size)
        lstm_out, _ = self.lstm(x)
        # Tomar última salida
        last_output = lstm_out[:, -1, :]
        out = self.fc(last_output)
        return out

# 2. GRU
class SimpleGRU(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=24, dropout=0.2):
        super(SimpleGRU, self).__init__()
        
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        gru_out, _ = self.gru(x)
        last_output = gru_out[:, -1, :]
        out = self.fc(last_output)
        return out

# 3. Bidirectional LSTM
class BidirectionalLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=24, dropout=0.2):
        super(BidirectionalLSTM, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # *2 porque es bidirectional
        self.fc = nn.Linear(hidden_size * 2, output_size)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_output = lstm_out[:, -1, :]
        out = self.fc(last_output)
        return out

# 4. LSTM con Attention
class LSTMWithAttention(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=24, dropout=0.2):
        super(LSTMWithAttention, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Attention
        self.attention = nn.Linear(hidden_size, 1)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # LSTM output
        lstm_out, _ = self.lstm(x)
        # lstm_out: (batch, seq_len, hidden_size)
        
        # Attention weights
        attention_weights = torch.softmax(self.attention(lstm_out), dim=1)
        # attention_weights: (batch, seq_len, 1)
        
        # Context vector (weighted sum)
        context = torch.sum(attention_weights * lstm_out, dim=1)
        # context: (batch, hidden_size)
        
        out = self.fc(context)
        return out

print("✅ Arquitecturas RNN definidas:")
print("   1. SimpleLSTM")
print("   2. SimpleGRU")
print("   3. BidirectionalLSTM")
print("   4. LSTMWithAttention")

## 8. Función de Entrenamiento

In [ ]:
def train_rnn_model(model, model_name, train_loader, val_loader, X_test, y_test, epochs=50, lr=0.001):
    """
    Entrena modelo RNN y trackea con MLflow
    """
    
    with mlflow.start_run(run_name=f"{model_name}_forecast"):
        
        # Log parámetros
        mlflow.log_param("model_type", model_name)
        mlflow.log_param("seq_length", seq_length)
        mlflow.log_param("pred_length", pred_length)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("learning_rate", lr)
        mlflow.log_param("batch_size", batch_size)
        
        # Optimizer y loss
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)
        
        # Training
        train_losses = []
        val_losses = []
        
        print(f"\n{'='*60}")
        print(f"Entrenando: {model_name}")
        print(f"{'='*60}")
        
        best_val_loss = float('inf')
        patience = 10
        patience_counter = 0
        
        for epoch in range(epochs):
            # Train
            model.train()
            train_loss = 0
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
            
            train_loss /= len(train_loader)
            train_losses.append(train_loss)
            
            # Validation
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for batch_X, batch_y in val_loader:
                    outputs = model(batch_X)
                    loss = criterion(outputs, batch_y)
                    val_loss += loss.item()
            
            val_loss /= len(val_loader)
            val_losses.append(val_loss)
            
            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                # Guardar mejor modelo
                torch.save(model.state_dict(), f'{model_name}_best.pth')
            else:
                patience_counter += 1
            
            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
            
            # Log métricas
            mlflow.log_metric("train_loss", train_loss, step=epoch)
            mlflow.log_metric("val_loss", val_loss, step=epoch)
            
            if patience_counter >= patience:
                print(f"Early stopping en epoch {epoch+1}")
                break
        
        # Cargar mejor modelo
        model.load_state_dict(torch.load(f'{model_name}_best.pth'))
        
        # Predicciones en test
        model.eval()
        with torch.no_grad():
            predictions = model(X_test).cpu().numpy()
        
        y_test_np = y_test.cpu().numpy()
        
        # Desnormalizar
        predictions_rescaled = scaler.inverse_transform(predictions.reshape(-1, 1)).reshape(predictions.shape)
        y_test_rescaled = scaler.inverse_transform(y_test_np.reshape(-1, 1)).reshape(y_test_np.shape)
        
        # Métricas
        mae = mean_absolute_error(y_test_rescaled.flatten(), predictions_rescaled.flatten())
        rmse = np.sqrt(mean_squared_error(y_test_rescaled.flatten(), predictions_rescaled.flatten()))
        r2 = r2_score(y_test_rescaled.flatten(), predictions_rescaled.flatten())
        mape = np.mean(np.abs((y_test_rescaled.flatten() - predictions_rescaled.flatten()) / y_test_rescaled.flatten())) * 100
        
        mlflow.log_metric("test_mae", mae)
        mlflow.log_metric("test_rmse", rmse)
        mlflow.log_metric("test_r2", r2)
        mlflow.log_metric("test_mape", mape)
        mlflow.log_metric("best_val_loss", best_val_loss)
        
        # Guardar modelo PyTorch
        mlflow.pytorch.log_model(model, "pytorch_model")
        
        # Visualizaciones
        # Training curves
        plt.figure(figsize=(10, 5))
        plt.plot(train_losses, label='Train Loss')
        plt.plot(val_losses, label='Val Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'{model_name} - Training Curves')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.savefig(f'{model_name}_training_curves.png', dpi=150, bbox_inches='tight')
        mlflow.log_artifact(f'{model_name}_training_curves.png')
        plt.close()
        
        # Predictions plot (primeras 5 secuencias)
        fig, axes = plt.subplots(5, 1, figsize=(14, 12))
        for i in range(5):
            axes[i].plot(y_test_rescaled[i], label='Real', linewidth=2)
            axes[i].plot(predictions_rescaled[i], label='Predicción', linestyle='--', linewidth=2)
            axes[i].set_title(f'Secuencia {i+1}')
            axes[i].set_ylabel('Demanda (MW)')
            axes[i].legend()
            axes[i].grid(True, alpha=0.3)
        
        axes[-1].set_xlabel('Hora')
        plt.tight_layout()
        plt.savefig(f'{model_name}_predictions.png', dpi=150, bbox_inches='tight')
        mlflow.log_artifact(f'{model_name}_predictions.png')
        plt.close()
        
        print(f"\n✅ {model_name} completado")
        print(f"   MAE: {mae:.2f} MW")
        print(f"   RMSE: {rmse:.2f} MW")
        print(f"   R²: {r2:.4f}")
        print(f"   MAPE: {mape:.2f}%")
        
        return {
            'model_name': model_name,
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'mape': mape,
            'best_val_loss': best_val_loss,
            'model': model
        }

## 9. Entrenar Todos los Modelos

In [ ]:
# Configuración común
hidden_size = 128
num_layers = 3
dropout = 0.3
epochs = 50
lr = 0.001

# Modelos a entrenar
models = {
    'SimpleLSTM': SimpleLSTM(hidden_size=hidden_size, num_layers=num_layers, 
                             output_size=pred_length, dropout=dropout).to(device),
    'SimpleGRU': SimpleGRU(hidden_size=hidden_size, num_layers=num_layers, 
                          output_size=pred_length, dropout=dropout).to(device),
    'BidirectionalLSTM': BidirectionalLSTM(hidden_size=hidden_size, num_layers=num_layers, 
                                           output_size=pred_length, dropout=dropout).to(device),
    'LSTMWithAttention': LSTMWithAttention(hidden_size=hidden_size, num_layers=num_layers, 
                                           output_size=pred_length, dropout=dropout).to(device)
}

# Entrenar
results = []
for model_name, model in models.items():
    result = train_rnn_model(
        model, model_name, 
        train_loader, val_loader, 
        X_test_tensor, y_test_tensor,
        epochs=epochs, lr=lr
    )
    results.append(result)

## 10. Comparación de Modelos

In [ ]:
# DataFrame de resultados
results_df = pd.DataFrame([{k: v for k, v in r.items() if k != 'model'} for r in results])

print("📊 COMPARACIÓN DE ARQUITECTURAS RNN")
print("=" * 80)
print(results_df.to_string(index=False))

# Visualizar comparación
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics = ['mae', 'rmse', 'r2', 'mape']
titles = ['MAE (MW)', 'RMSE (MW)', 'R² Score', 'MAPE (%)']

for idx, (ax, metric, title) in enumerate(zip(axes.flat, metrics, titles)):
    results_df.plot(x='model_name', y=metric, kind='bar', ax=ax, legend=False, color='steelblue')
    ax.set_title(title)
    ax.set_xlabel('')
    ax.set_ylabel(title.split('(')[0].strip())
    ax.tick_params(axis='x', rotation=45)
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('rnn_models_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# Mejor modelo
best_idx = results_df['mae'].idxmin()
best_model_name = results_df.loc[best_idx, 'model_name']
best_model = results[best_idx]['model']

print(f"\n🏆 Mejor modelo: {best_model_name}")
print(f"   MAE: {results_df.loc[best_idx, 'mae']:.2f} MW")
print(f"   RMSE: {results_df.loc[best_idx, 'rmse']:.2f} MW")
print(f"   R²: {results_df.loc[best_idx, 'r2']:.4f}")
print(f"   MAPE: {results_df.loc[best_idx, 'mape']:.2f}%")

## 11. Export ONNX del Mejor Modelo

In [ ]:
print(f"🔄 Exportando {best_model_name} a ONNX...\n")

# Preparar modelo para export
best_model.eval()

# Input de ejemplo
dummy_input = torch.randn(1, seq_length, 1).to(device)

# Export a ONNX
onnx_filename = f"{best_model_name}_timeseries.onnx"

torch.onnx.export(
    best_model,
    dummy_input,
    onnx_filename,
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)

print(f"✅ Modelo ONNX guardado: {onnx_filename}")

# Verificar modelo ONNX
onnx_model = onnx.load(onnx_filename)
onnx.checker.check_model(onnx_model)
print("✅ Modelo ONNX verificado")

# Test de inferencia ONNX
print("\n🧪 Testeando inferencia ONNX...")

ort_session = rt.InferenceSession(onnx_filename)

# Comparar predicciones
test_input = X_test_tensor[:5].cpu().numpy()

# PyTorch
with torch.no_grad():
    pytorch_pred = best_model(torch.FloatTensor(test_input).to(device)).cpu().numpy()

# ONNX
onnx_pred = ort_session.run(None, {'input': test_input.astype(np.float32)})[0]

# Comparar
diff = np.abs(pytorch_pred - onnx_pred).max()
print(f"   Diferencia máxima: {diff:.8f}")
print(f"   Predicciones coinciden: {'✅ Sí' if diff < 1e-5 else '❌ No'}")

# Benchmark
print("\n⚡ Benchmark de inferencia (100 iteraciones):")

n_iterations = 100
test_batch = X_test_tensor[:32]

# PyTorch
start = time.time()
with torch.no_grad():
    for _ in range(n_iterations):
        _ = best_model(test_batch)
pytorch_time = time.time() - start

# ONNX
test_batch_np = test_batch.cpu().numpy()
start = time.time()
for _ in range(n_iterations):
    _ = ort_session.run(None, {'input': test_batch_np.astype(np.float32)})
onnx_time = time.time() - start

print(f"   PyTorch: {pytorch_time:.4f}s ({pytorch_time/n_iterations*1000:.2f}ms/iter)")
print(f"   ONNX: {onnx_time:.4f}s ({onnx_time/n_iterations*1000:.2f}ms/iter)")
print(f"   Speedup: {pytorch_time/onnx_time:.2f}x")

## 12. Predicción Multi-Step con Mejor Modelo

In [ ]:
def multi_step_forecast(model, initial_sequence, n_steps, scaler):
    """
    Predicción multi-step iterativa
    """
    model.eval()
    predictions = []
    
    current_seq = initial_sequence.clone()
    
    with torch.no_grad():
        for _ in range(n_steps):
            # Predecir próximos 24 pasos
            pred = model(current_seq.unsqueeze(0))
            predictions.append(pred.cpu().numpy()[0])
            
            # Actualizar secuencia (usar últimos 24 valores predichos)
            current_seq = torch.cat([
                current_seq[pred_length:],
                pred.unsqueeze(-1)
            ], dim=0)
    
    predictions = np.array(predictions).reshape(-1)
    predictions_rescaled = scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()
    
    return predictions_rescaled

# Hacer predicción de 7 días (7 * 24 horas)
print("🔮 Predicción multi-step: 7 días (168 horas)\n")

initial_seq = X_test_tensor[0]
n_days = 7
n_steps_forecast = n_days  # Cada step predice 24 horas

multi_step_preds = multi_step_forecast(best_model, initial_seq, n_steps_forecast, scaler)

# Obtener datos reales correspondientes
start_idx = train_size + val_size + seq_length
end_idx = start_idx + (n_days * pred_length)
real_values = scaler.inverse_transform(energy_scaled[start_idx:end_idx].reshape(-1, 1)).flatten()

# Visualizar
hours = np.arange(len(multi_step_preds))
days_labels = [f"Día {i//24 + 1}" if i % 24 == 0 else "" for i in hours]

plt.figure(figsize=(16, 6))
plt.plot(hours, real_values[:len(multi_step_preds)], label='Real', linewidth=2, alpha=0.7)
plt.plot(hours, multi_step_preds, label='Predicción', linestyle='--', linewidth=2)
plt.xlabel('Horas')
plt.ylabel('Demanda de Energía (MW)')
plt.title(f'Predicción Multi-Step: {n_days} días ({best_model_name})')
plt.legend()
plt.grid(True, alpha=0.3)

# Marcar días
for day in range(1, n_days + 1):
    plt.axvline(x=day*24, color='gray', linestyle=':', alpha=0.5)
    plt.text(day*24 - 12, plt.ylim()[1]*0.95, f'Día {day}', ha='center')

plt.tight_layout()
plt.savefig('multi_step_forecast.png', dpi=150, bbox_inches='tight')
plt.show()

# Métricas multi-step
mae_multi = mean_absolute_error(real_values[:len(multi_step_preds)], multi_step_preds)
rmse_multi = np.sqrt(mean_squared_error(real_values[:len(multi_step_preds)], multi_step_preds))

print(f"\n📊 Métricas de predicción multi-step ({n_days} días):")
print(f"   MAE: {mae_multi:.2f} MW")
print(f"   RMSE: {rmse_multi:.2f} MW")

## 13. Resumen y Conclusiones

In [ ]:
print("🎉 RESUMEN: RNN PARA SERIES TEMPORALES")
print("=" * 70)

print(f"\n📊 DATOS:")
print(f"   Total registros: {len(df):,} horas ({len(df)/(24*365):.1f} años)")
print(f"   Secuencia de entrada: {seq_length} horas (1 semana)")
print(f"   Predicción: {pred_length} horas (1 día)")
print(f"   Train: {len(X_train):,}, Val: {len(X_val):,}, Test: {len(X_test):,}")

print(f"\n🤖 ARQUITECTURAS EVALUADAS: {len(results)}")
for r in results:
    print(f"   - {r['model_name']}: MAE={r['mae']:.2f} MW, R²={r['r2']:.4f}")

print(f"\n🏆 MEJOR MODELO: {best_model_name}")
print(f"   MAE: {results_df.loc[best_idx, 'mae']:.2f} MW")
print(f"   RMSE: {results_df.loc[best_idx, 'rmse']:.2f} MW")
print(f"   R²: {results_df.loc[best_idx, 'r2']:.4f}")
print(f"   MAPE: {results_df.loc[best_idx, 'mape']:.2f}%")

print(f"\n✅ EXPORTACIÓN ONNX:")
print(f"   Archivo: {onnx_filename}")
print(f"   Verificado: Sí")
print(f"   Speedup: {pytorch_time/onnx_time:.2f}x")

print(f"\n💡 MEJORES PRÁCTICAS:")
print(f"   ✅ Usa secuencias largas para capturar patrones (1 semana+)")
print(f"   ✅ Normaliza datos antes de entrenar")
print(f"   ✅ Implementa early stopping")
print(f"   ✅ Compara múltiples arquitecturas")
print(f"   ✅ Bidirectional LSTM captura contexto pasado/futuro")
print(f"   ✅ Attention ayuda a enfocarse en partes relevantes")
print(f"   ✅ Export a ONNX para producción")
print(f"   ✅ DuckDB acelera procesamiento de datos")

print(f"\n🚀 CASOS DE USO:")
print(f"   - Predicción de demanda energética")
print(f"   - Forecasting de ventas")
print(f"   - Predicción de tráfico")
print(f"   - Análisis de series financieras")
print(f"   - Mantenimiento predictivo")

con.close()
print("\n✅ Conexión DuckDB cerrada")
print("\n💻 Ver resultados: mlflow ui --port 5000")
print("\n" + "=" * 70)